In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [73]:
%%capture
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol']=sp500['Symbol'].str.replace('.','-')
symbols_list=sp500['Symbol'].unique().tolist()
end_date='2024-08-01'
start_date=pd.to_datetime(end_date)-pd.DateOffset(365*8)
df=yf.download(tickers=symbols_list,
               start=start_date,
               end=end_date).stack()
print(df)


In [75]:
df.index.names = ['Date', 'Ticker']

In [76]:
df.columns

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

In [77]:
%%capture
#Calculate the log returns for the stocks
def calculate_log_returns(df,column='Adj Close'):
    return np.log(df[column]/df[column].shift(1))

# log_returns=np.log(df_1/df_1.shift(1)).dropna()
# log_returns=log_returns.dropna()
# log_returns

#Calculate momentum based on log returns
def calculate_momentum_log_returns(df, period=126):
    log_retuns=np.log(df/df.shift(1))
    return log_returns.rolling(window=period).sum().iloc[-1]

In [78]:
%%capture
#Calculate log returns and momentum
df['log_returns']=calculate_log_returns(df)
df['momentum_returns']=calculate_momentum_log_returns(df)
print("Log Returns:")
print(log_returns.tail())

In [80]:
#calculate the rsi
# Function to calculate RSI
def calculate_rsi(df, column='Adj Close', period=14):
    # Calculate price changes
    delta = df[column].diff()

    # Calculate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate rolling averages
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    # Calculate RS and RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Add RSI to the DataFrame
df['RSI'] = calculate_rsi(df)

In [82]:
momentum_results = []
dates = []

#every 6 months
date_range=pd.date_range(start='2022-08-01',end='2024-08-01',freq='6M')
for date in date_range:
    if date not in df.index:
        continue
    end_date1=date
    start_date1=end_date1-pd.DateOffset(days=126)

    period_data=df.loc[start_date1:end_date1]

    if period_data.shape[0]<126:
        continue

    momentum1=calculate_momentum_log_returns(period_data)
    momentum_results.append(momentum1)
    dates.append(end_date1)

momentum_df = pd.DataFrame(momentum_results, index=dates)    

latest_momentum = momentum_df.iloc[-1]
ranked_momentum = latest_momentum.rank(ascending=False)

# Select the top 5 stocks based on the latest momentum
top_stocks = ranked_momentum.nsmallest(5).index

top_stocks
    

Index(['A', 'AAL', 'AAPL', 'ABBV', 'ABNB'], dtype='object', name='Ticker')

In [90]:
# Calculate the allocation (equal weight to each top stock)
allocation = 1 / len(top_stocks)
portfolio = pd.DataFrame({'Ticker': top_stocks, 'Allocation': allocation})

# Print the portfolio
print("\nPortfolio Allocation:")
print(portfolio)


Portfolio Allocation:
                        Ticker  Allocation
0     (2016-08-03 00:00:00, A)         0.2
1   (2016-08-03 00:00:00, AAL)         0.2
2  (2016-08-03 00:00:00, AAPL)         0.2
3  (2016-08-03 00:00:00, ABBV)         0.2
4   (2016-08-03 00:00:00, ABT)         0.2
